In [ ]:
%load_ext autoreload
%autoreload 2
import poisson_atac as patac
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os

sns.set_style("whitegrid")

fig_path = '/storage/groups/ml01/workspace/laura.martens/atac_poisson_data/panels/Figure1'

def plot_df(df, figsize=(15, 8), xlabel="Number of reads"):
    df["color"]= df["#reads"]>0
    fig, ax = plt.subplots(figsize=figsize)
    g = sns.barplot(data=df, x = '#reads', y = 'count', hue="color", palette="Paired", ax = ax, dodge=False)
    g.set_yscale('log')
    labels = g.get_xticklabels()
    labels[1::2] = np.repeat('', len(labels[1::2]))
    g.set_xticklabels(labels, rotation=90);
    g.legend().set_visible(False)
    g.grid(False)
    plt.xlabel(xlabel)
    plt.tight_layout()

def mean_var_scatter(df, label=r"$\sigma^2=\mu$", endpoint=(1,1), linestyle="solid"):
    fig, ax = plt.subplots(figsize=(6,6))
    g=sns.scatterplot(data=df, x="mean", y ="variance")
    g.set_yscale('log')
    g.set_xscale('log')
    
    if isinstance(label, list):
        for i, l in enumerate(label):
            g.axline((0,0), endpoint[i], color='darkgrey', linestyle=linestyle[i], label=l)
    else:
        g.axline((0,0), endpoint, color='darkgrey', linestyle=linestyle, label=label)
    g.grid(":", color="lightgrey")
    plt.xlabel(r"mean $\mu$")
    plt.ylabel(r"variance $\sigma^2$")
    plt.legend()
    plt.tight_layout()

def plot_mean_variance(adata, save_path=None, prefix=""):

    # number of counts
    X = np.ceil(adata.layers["counts"].A/2)
    mean_counts = X.mean(axis = 0)
    var_counts = X.var(axis = 0)
    df = pd.DataFrame({'mean': mean_counts, 'variance': var_counts})
    
    mean_var_scatter(df, label="Shot noise limit", endpoint=(1,1))
    if save_path:
        plt.gcf().savefig(os.path.join(save_path, f"{prefix}_raw_data_scatter.png"))
        plt.gcf().savefig(os.path.join(save_path, f"{prefix}_raw_data_scatter.pdf"))
    
    X = adata.layers["counts"].A
    mean_counts = X.mean(axis = 0)
    var_counts = X.var(axis = 0)
    df = pd.DataFrame({'mean': mean_counts, 'variance': var_counts})
    
    mean_var_scatter(df, label=["Shot noise limit", r"$\sigma^2=2\mu$"], endpoint=[(1,1),(1,2)], linestyle=["solid", ":"])
    if save_path:
        plt.gcf().savefig(os.path.join(save_path, f"{prefix}_transformed_data_scatter.png"))
        lt.gcf().savefig(os.path.join(save_path, f"{prefix}_transformed_data_scatter.pdf"))

5

def plot_dataset(adata, save_path=None, prefix=None, limit=None):
    counts = pd.Series(adata.layers["counts"].data).value_counts().to_frame().reset_index().rename({0: 'count', 'index':'#reads'}, axis = 1)
    sns_df = pd.concat([counts, pd.DataFrame({'#reads':0.0 , 'count':adata.shape[0]*adata.shape[1]-len(adata.layers["counts"].data) }, index=[0])])
    sns_df['#reads'] = pd.Categorical(sns_df['#reads'].astype(int), categories=np.arange(sns_df['#reads'].max() + 1).astype(int), ordered=True)

    ## Plot all raw counts

    plot_df(sns_df, figsize = (40,8))
    if save_path:
        plt.gcf().savefig(os.path.join(save_path, f"{prefix}_raw_data_dist.pdf"))
        plt.gcf().savefig(os.path.join(save_path, f"{prefix}_raw_data_dist.png"))
    
    ## Plot all raw counts in a range of 0-80
    if limit:
        sns_df = sns_df[sns_df['#reads']<=limit]
        sns_df["#reads"].cat.set_categories(np.arange(limit).astype(int), inplace=True)

    plot_df(sns_df, figsize = (10,4))
    if save_path:
        plt.gcf().savefig(os.path.join(save_path, f"{prefix}_raw_data_dist_zoom.pdf"))
        plt.gcf().savefig(os.path.join(save_path, f"{prefix}_raw_data_dist_zoom.png"))
    ## Convert counts

    data = np.ceil(adata.layers["counts"].data/2)

    counts = pd.Series(data).value_counts().to_frame().reset_index().rename({0: 'count', 'index':'#reads'}, axis = 1)
    sns_df = pd.concat([counts, pd.DataFrame({'#reads':0.0 , 'count':adata.shape[0]*adata.shape[1]-len(adata.layers["counts"].data) }, index=[0])])
    sns_df['#reads'] = pd.Categorical(sns_df['#reads'].astype(int), categories=np.arange(sns_df['#reads'].max() + 1).astype(int), ordered=True)

    plot_df(sns_df, figsize = (40,8), xlabel="Number of fragments")
    if save_path:
        plt.gcf().savefig(os.path.join(save_path, f"{prefix}_transformed_data_dist.pdf"))
        plt.gcf().savefig(os.path.join(save_path, f"{prefix}_transformed_data_dist.png"))
    ## Plot all raw counts in a range of 0-80
    if limit:
        sns_df = sns_df[sns_df['#reads']<=limit]
        sns_df["#reads"].cat.set_categories(np.arange(limit).astype(int), inplace=True)

    plot_df(sns_df, figsize = (10,4), xlabel="Number of fragments")
    if save_path:
        plt.gcf().savefig(os.path.join(save_path, f"{prefix}_transformed_data_dist_zoom.pdf"))
        plt.gcf().savefig(os.path.join(save_path, f"{prefix}_transformed_data_dist_zoom.png"))

def plot_proportions(adata, figsize, fontsize, save_path=None, prefix="", lower_limit=0, upper_limit=2):
    
    # From scvelo https://github.com/theislab/scvelo/blob/master/scvelo/plotting/proportions.py
    counts = pd.Series(adata.layers["counts"].data).value_counts().to_frame().reset_index().rename({0: 'count', 'index':'reads'}, axis = 1)
    counts = pd.concat([counts, pd.DataFrame({'reads': 0.0 , 'count':adata.shape[0]*adata.shape[1]-len(adata.layers["counts"].data) }, index=[0])])
    counts["reads"] = counts["reads"].astype(int)
    
    sns_df = pd.concat([counts[(counts["reads"] <= upper_limit) & (counts["reads"] > lower_limit)], pd.DataFrame({'reads': f">{upper_limit}", "count": counts["count"][counts["reads"]>upper_limit].sum()}, index =[0])])
    
    fig, ax = plt.subplots(figsize=figsize, dpi=100)
    colors = sns.color_palette("Paired", sns_df.shape[0], as_cmap=False)
    autopct = "%1.1f%%" 
    #explode=(0, 0.1, 0.1, 0.1)
    explode=None
    pie = ax.pie(
        sns_df["count"],
        colors=colors,
        explode=explode,
        autopct=autopct,
        shadow=False,
        startangle=45,
        textprops={'size': 'smaller'}
    )
    if autopct is not None:
        for pct, color in zip(pie[-1], colors):
            r, g, b = color
            pct.set_color("black" if r * g * b < 0.5 else "darkgrey")
            #plt.setp(pct, size='large')
            #pct.set_fontweight("bold")
            pct.set_fontsize(fontsize)
    ax.legend(
        sns_df["reads"].astype(str) + " read(s)",
        ncol=len(sns_df["reads"]),
        bbox_to_anchor=(0, 1),
        loc="lower left",
        fontsize=fontsize,
    )
    if save_path:
        fig.savefig(os.path.join(save_path, f"{prefix}_proportions.png"))
        fig.savefig(os.path.join(save_path, f"{prefix}_proportions.pdf"))

# Neurips

adata = patac.data.load_neurips(only_train=False, convert_counts = False)

adata

patac.pl.count_distribution(adata, layer="counts", figsize = (12, 5), limit = 80)

patac.pl.mean_variance(adata, layer="counts", figsize=(6,6))

patac.pl.proportions(adata, lower_limit=1, upper_limit=2, layer="counts")

plot_mean_variance(adata, save_path=fig_path, prefix="neurips")

plot_proportions(adata, figsize=(5,5), fontsize=12) #, save_path=fig_path, prefix="neurips"

# Hematopoeisis

adata = patac.data.load_hematopoiesis(convert_counts = False)

plot_dataset(adata, save_path=fig_path, prefix="hematopoiesis", limit=None)

plot_mean_variance(adata, save_path=fig_path, prefix="hematopoiesis")

plot_proportions(adata, figsize=(5,5), fontsize=14, save_path=fig_path, prefix="hematopoiesis")